## Implementation of Brunel 2000 using the Hybrid LFP scheme
* Hybrid LFP scheme example script, applying the methodology with a model
implementation similar to:

* Nicolas Brunel. "Dynamics of Sparsely Connected Networks of Excitatory and
Inhibitory Spiking Neurons". J Comput Neurosci, May 2000, Volume 8,
Issue 3, pp 183-208

* Synopsis of the main simulation procedure:
1. Loading of parameterset
    a. network parameters
    b. parameters for hybrid scheme
2. Set up file destinations for different simulation output
3. network simulation
    a. execute network simulation using NEST (www.nest-initiative.org)
    b. merge network output (spikes, currents, voltages)
4. Create a object-representation that uses sqlite3 of all the spiking output
5. Iterate over post-synaptic populations:
    a. Create Population object with appropriate parameters for
       each specific population
    b. Run all computations for populations
    c. Postprocess simulation output of all cells in population
6. Postprocess all cell- and population-specific output data
7. Create a tarball for all non-redundant simulation output

* recommend use mpirun for parallel execution

In [11]:
from example_plotting import *
import brunel_alpha_nest as BN
import lfpykit
from mpi4py import MPI
import h5py
from parameters import ParameterSet
from hybridLFPy import PostProcess, Population, CachedNetwork, setup_file_dest
import neuron  # NEURON compiled with MPI must be imported before NEST/mpi4py
from time import time
from matplotlib import gridspec
import matplotlib.pyplot as plt
import os
import numpy as np
if 'DISPLAY' not in os.environ:
    import matplotlib
    matplotlib.use('Agg')
import matplotlib.style

matplotlib.style.use('classic')


In [12]:

########## matplotlib settings ###########################################
plt.close('all')
plt.rcParams.update({'figure.figsize': [10.0, 8.0]})


# set some seed values
SEED = 12345678
SIMULATIONSEED = 12345678
np.random.seed(SEED)


In [13]:

################# Initialization of MPI stuff ############################
COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()

# if True, execute full model. If False, do only the plotting.
# Simulation results must exist.
properrun = True


In [14]:

# check if mod file for synapse model specified in alphaisyn.mod is loaded
if not hasattr(neuron.h, 'AlphaISyn'):
    if RANK == 0:
        os.system('nrnivmodl')
    COMM.Barrier()
    neuron.load_mechanisms('.')


##########################################################################
# PARAMETERS
##########################################################################

# Set up parameters using the NeuroTools.parameters.ParameterSet class.

# Access parameters defined in example script implementing the network using
# pynest, brunel_alpha_nest.py, adapted from the NEST v2.4.1 release. This will
# not execute the network model, but see below.


/home/hyunsu/miniconda3/envs/lfpy/bin/nrnivmodl:10: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import working_set


/home/storage2/hyunsu_DATA/GitHub_Active/LFPy_practice/Examples_from_hybridLFPy
Mod files:

 -> Compiling mod_func.cpp
 => LINKING shared library ./libnrnmech.so
 => LINKING executable ./special LDFLAGS are:    -pthread
Mechanisms already loaded from path: ..  Aborting.


/home/hyunsu/miniconda3/envs/lfpy/bin/../lib/gcc/x86_64-conda-linux-gnu/12.4.0/../../../../x86_64-conda-linux-gnu/bin/ld: /home/hyunsu/miniconda3/envs/lfpy/lib/python3.12/site-packages/neuron/.data/lib/libnrniv.so: undefined reference to `memcpy@GLIBC_2.14'
collect2: error: ld returned 1 exit status
make: *** [/home/hyunsu/miniconda3/envs/lfpy/lib/python3.12/site-packages/neuron/.data/bin/nrnmech_makefile:125: special] Error 1
Traceback (most recent call last):
  File "/home/hyunsu/miniconda3/envs/lfpy/bin/nrnivmodl", line 114, in <module>
    subprocess.check_call([exe, *sys.argv[1:]])
  File "/home/hyunsu/miniconda3/envs/lfpy/lib/python3.12/subprocess.py", line 413, in check_call
    raise CalledProcessError(retcode, cmd)
subprocess.CalledProcessError: Command '['/home/hyunsu/miniconda3/envs/lfpy/lib/python3.12/site-packages/neuron/.data/bin/nrnivmodl']' returned non-zero exit status 2.


In [15]:

# set up file destinations differentiating between certain output
PS = ParameterSet(dict(
    # Main folder of simulation output
    savefolder='simulation_output_example_brunel',

    # make a local copy of main files used in simulations
    sim_scripts_path=os.path.join('simulation_output_example_brunel',
                                  'sim_scripts'),

    # destination of single-cell output during simulation
    cells_path=os.path.join('simulation_output_example_brunel', 'cells'),

    # destination of cell- and population-specific signals, i.e., compund LFPs,
    # CSDs etc.
    populations_path=os.path.join('simulation_output_example_brunel',
                                  'populations'),

    # location of spike output from the network model
    spike_output_path=BN.spike_output_path,

    # destination of figure file output generated during model execution
    figures_path=os.path.join('simulation_output_example_brunel', 'figures')
))

In [16]:

# population (and cell type) specific parameters
PS.update(dict(
    # no cell type specificity within each E-I population
    # hence X == x and Y == X
    X=["EX", "IN"],

    # population-specific LFPy.Cell parameters
    cellParams=dict(
        # excitory cells
        EX=dict(
            morphology='morphologies/ex.swc',
            v_init=BN.neuron_params['E_L'],
            cm=1.0,
            Ra=150,
            passive=True,
            passive_parameters=dict(
                g_pas=1. / (BN.neuron_params['tau_m'] * 1E3),  # assume cm=1
                e_pas=BN.neuron_params['E_L']),
            nsegs_method='lambda_f',
            lambda_f=100,
            dt=BN.dt,
            tstart=0,
            tstop=BN.simtime,
            verbose=False,
        ),
        # inhibitory cells
        IN=dict(
            morphology='morphologies/in.swc',
            v_init=BN.neuron_params['E_L'],
            cm=1.0,
            Ra=150,
            passive=True,
            passive_parameters=dict(
                g_pas=1. / (BN.neuron_params['tau_m'] * 1E3),  # assume cm=1
                e_pas=BN.neuron_params['E_L']),
            nsegs_method='lambda_f',
            lambda_f=100,
            dt=BN.dt,
            tstart=0,
            tstop=BN.simtime,
            verbose=False,
        )),

    # assuming excitatory cells are pyramidal
    rand_rot_axis=dict(
        EX=['z'],
        IN=['x', 'y', 'z'],
    ),


    # kwargs passed to LFPy.Cell.simulate().
    # It can be empty, but if `rec_imem=True`, the signal predictions will be
    # performed using recorded transmembrane currents.
    simulationParams=dict(rec_imem=True),

    # set up parameters corresponding to cylindrical model populations
    populationParams=dict(
        EX=dict(
            number=BN.NE,
            radius=100,
            z_min=-450,
            z_max=-350,
            min_cell_interdist=1.,
            min_r=[[-1E199, -600, -550, 1E99], [0, 0, 10, 10]],
        ),
        IN=dict(
            number=BN.NI,
            radius=100,
            z_min=-450,
            z_max=-350,
            min_cell_interdist=1.,
            min_r=[[-1E199, -600, -550, 1E99], [0, 0, 10, 10]],
        ),
    ),

    # set the boundaries between the "upper" and "lower" layer
    layerBoundaries=[[0., -300],
                     [-300, -500]],

    # set the geometry of the virtual recording device
    electrodeParams=dict(
        # contact locations:
        x=[0] * 6,
        y=[0] * 6,
        z=[x * -100. for x in range(6)],
        # extracellular conductivity:
        sigma=0.3,
        # contact surface normals, radius, n-point averaging
        N=[[1, 0, 0]] * 6,
        r=5,
        n=20,
        seedvalue=None,
        # dendrite line sources, soma as sphere source (Linden2014)
        method='root_as_point',
    ),

    # parameters for LFPykit.LaminarCurrentSourceDensity
    CSDParams=dict(
        z=np.array([[-(i + 1) * 100, -i * 100] for i in range(6)]) + 50.,
        r=np.ones(6) * 100  # same as population radius
    ),

    # runtime, cell-specific attributes and output that will be stored
    savelist=[],

    # time resolution of saved signals
    dt_output=1.
))

ValueError: not enough values to unpack (expected 2, got 1)